In [24]:
import numpy as np

class T:
    def __init__(self, data, children=()):
        if isinstance(data, T):
            raise
        data = np.asarray(data)
            
        self.data = data
        self.grad = np.zeros_like(data, dtype=float)
        self._backward = None

    @property
    def shape(self):
        return self.data.shape

    def __repr__(self):
        return f'T({self.data})'

    def __mul__(self, other):
        if not isinstance(other, T):
            other = T(other)

        t = T(self.data * other.data, children=(self, other))

        def _backward():
            # update self.grad
            b = np.broadcast(self.data, other.data)
            grad = np.empty(b.shape)
            grad.flat = [o for (s,o) in b]
            self.grad += grad * t.grad

            # update other.grad
            b = np.broadcast(self.data, other.data)
            grad = np.empty(b.shape)
            grad.flat = [s for (s,o) in b]
            other.grad += grad * t.grad
    

        t._backward = _backward
                
        return t
                
    def __add__(self, other):
        if not isinstance(other, T):
            other = T(other)

        t = T(self.data + other.data, children=(self, other))

        def _backward():
            # update self.grad
            b = np.broadcast(self.data, other.data)
            grad = np.ones(b.shape)
            self.grad += grad * t.grad

            # update other.grad
            b = np.broadcast(self.data, other.data)
            grad = np.ones(b.shape)
            other.grad += grad * t.grad
    

        t._backward = _backward
                
        return t

    def __pow__(self, other):
        assert isinstance(other, (int,float))
        exp = other

        t = T(self.data ** other, children=(self,))   

        def _backward():
            b = np.broadcast(self.data, exp)
            grad = np.empty(b.shape)
            grad.flat = [e*s**(e-1) for (s,e) in b]
            
            self.grad += grad * t.grad
        t._backward = _backward
        return t
        

    def __truediv__(self, other):
        return self * other ** -1

    def backward(self):
        self.grad
        self._backward()

    
w1 = T([[1,2]])
b = T(5)
L = w1 * 2 + b
# L.grad
L.backward()


ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (1,2)

In [39]:
list(np.nditer([x,y]))

[(array(1), array(3)), (array(2), array(3))]

In [103]:
a = np.ones((2,3))*2
b = np.ones((3,))*3
r = None

with np.nditer([a,b, None],flags=['multi_index']) as it:
    for x, y,z in it:
        print(it.multi_index)
        z[...] = z
        r = it.operands[2]

(0, 0)
(0, 1)
(0, 2)
(1, 0)
(1, 1)
(1, 2)


In [80]:
r

array([[3., 3., 3.],
       [3., 3., 3.]])

In [82]:
a_ = torch.tensor(a, requires_grad=True)
b_ = torch.tensor(b, requires_grad=True)

In [95]:
b_.grad

tensor([4., 4., 4.], dtype=torch.float64)

In [86]:
l = a_*b_
l.sum().backward()

In [66]:
x = np.ones((2,3))
y = np.ones((3,))*2
it = np.nditer([x,y], flags=['reduce_ok','multi_index'], op_flags=[['readonly'],['readonly']])
for x in it:
    print(x)
    # print(it.index)
    print(it.multi_index)

(array(1.), array(2.))
(0, 0)
(array(1.), array(2.))
(0, 1)
(array(1.), array(2.))
(0, 2)
(array(1.), array(2.))
(1, 0)
(array(1.), array(2.))
(1, 1)
(array(1.), array(2.))
(1, 2)


ValueError: Iterator is past the end

In [42]:
def square(a,b):
    with np.nditer([a, b]) as it:
        # for x, y in it:
        #     y[...] = 1
        print(it.operands)
        return it.operands[1]

square([1,2],2)

(array([1, 2]), array(2))


array(2)

In [29]:
x = np.array([1,2])
y = np.array(3)
b = np.broadcast(x,y)
b.index

0

In [36]:
list(b.iters[1])

[]

In [31]:
list(b)

[(1, 3), (2, 3)]

In [25]:
%debug

> /var/folders/nw/lktt921x1wg_wt5tfy5q9k2c0000gn/T/ipykernel_37031/1999081766.py(59)_backward()
     57             b = np.broadcast(self.data, other.data)
     58             grad = np.ones(b.shape)
---> 59             other.grad += grad * t.grad
     60 
     61 



ipdb>  other.grad


array(0.)


ipdb>  self.grad


array([[0., 0.]])


...

ipdb>  self.data.shape


(1, 2)


ipdb>  other.data.shape


()


ipdb>  q


In [21]:
# _get_grads(x,w1,b, 'Value')

In [22]:
import torch

def _get_grads(x,w1,b1,backend):
    if backend == 'Value':
        f = T
        # exp_ = exp
        # log_ = log
    elif backend == 'torch':
        f = torch.tensor
        exp_ = torch.exp
        log_ = torch.log
    else:
        raise ValueError(f'{backend} is invalid')

    x = f(x)
    w1 = f(w1)
    # b1 = f(b1)
    L = x*w1

    L.backward()
    return x.grad, w1.grad
    

x = [1,2,3]
w1 = [.1,.1,.1]
b1 = 3.

# assert np.isclose(_get_grads(x,w1,b, 'Value'), _get_grads(x,w1,b, 'torch')).all()
_get_grads(x,w1,b, 'Value')

(array([0., 0., 0.]), array([0., 0., 0.]))

In [12]:
%debug

> /var/folders/nw/lktt921x1wg_wt5tfy5q9k2c0000gn/T/ipykernel_37031/651032152.py(6)__init__()
      4     def __init__(self, data, children=()):
      5         if isinstance(data, T):
----> 6             raise
      7         data = np.asarray(data)
      8 



ipdb>  q


In [79]:
# np.empty((20,10))

In [67]:
L.grad = 1
L._backward()

In [68]:
b.grad

array([[1., 1.]])

In [62]:
# w1.grad

In [29]:
w1.data

array([1, 2])

In [27]:
w1.grad

2

In [14]:
np.array(5).shape

()